In [2]:
from keras.utils import np_utils
from keras.layers import Input,Dense,BatchNormalization
from keras import regularizers
from keras.models import Model
import numpy as np
import pandas as pd
import tensorflow
from keras.callbacks import TensorBoard



Using TensorFlow backend.


In [3]:
path = '/home/niks/projects/datasets/ML_datasets'
def get_file(train=False):
    if train:return path + '/mnist_train.csv'
    else : return path + '/mnist_test.csv'

In [4]:
train_data = pd.read_csv(get_file(True))
test_data = pd.read_csv(get_file(False))

In [5]:
train_y = train_data['5']
train_x = train_data.loc[:,'0':]
test_x = test_data.loc[:,'0':]
test_y = test_data['7']

train_x,test_x = map(lambda x:np.array(x),[train_x,test_x])
test_y,train_y = map(lambda x:np.array(x).reshape(x.shape[0],1),[test_y,train_y])
print(train_x.shape)
train_x = (train_x / 255)
test_x = (test_x / 255)
train_y = np_utils.to_categorical(train_y,10)
test_y = np_utils.to_categorical(test_y,10)

(59999, 784)


In [6]:
from sklearn.model_selection import train_test_split
train_x,train_dev_x,train_y,train_dev_y = train_test_split(train_x, train_y,train_size=0.9,test_size=0.1,random_state=0,shuffle=True)
test_x,dev_x,test_y,dev_y = train_test_split(test_x, test_y, random_state=0,shuffle=True)

In [7]:
print("Train_x--> {}\t\t Train_y {} \nTrain_dev_x -> {} \t Train_dev_y -> {} \nTest_x -> {} \t\t Test_y -> {} \nDev_x -> {} \t\t Dev_y -> {}".format(train_x.shape,train_y.shape,train_dev_x.shape,train_dev_y.shape,test_x.shape,test_y.shape,dev_x.shape,dev_y.shape))

Train_x--> (53999, 784)		 Train_y (53999, 10) 
Train_dev_x -> (6000, 784) 	 Train_dev_y -> (6000, 10) 
Test_x -> (7499, 784) 		 Test_y -> (7499, 10) 
Dev_x -> (2500, 784) 		 Dev_y -> (2500, 10)


In [62]:
from keras.layers import Input,Dense,BatchNormalization
from keras import regularizers
from keras.models import Model
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.optimizers import Adagrad
K.clear_session()

inputs = Input(shape=(784,))

X = Dense(units=30,activation='relu',name='Input')(inputs)

X = BatchNormalization(name='batch_normal1')(X)

X = Dense(units=40,activation='relu',kernel_regularizer=regularizers.l1_l2(l1=0.0001, l2=0.0001),name='dense1')(X)

X = BatchNormalization(name='batch_normal2')(X)

X = Dense(units=80,activation='relu',kernel_regularizer=regularizers.l1_l2(l1=0.0001, l2=0.0001),name='dense2')(X)

X = BatchNormalization(name='batch_normal3')(X)

X = Dense(units=40,activation='relu',kernel_regularizer=regularizers.l1_l2(l1=0.0001, l2=0.0001),name='dense3')(X)


prediction = Dense(units=10,activation='softmax',name='output')(X)

model = Model(inputs=inputs,outputs=prediction)

model.compile(optimizer=Adagrad(lr=0.1, epsilon=None, decay=0.00000002)
,loss='categorical_crossentropy',metrics = ['accuracy'])

# log_dir = '/tmp/log/'
# tfk = TensorBoard(log_dir=log_dir, histogram_freq=0,write_graph=True, write_images=True)
import shutil
shutil.rmtree('/tmp/log/training')
shutil.rmtree('/tmp/log/validation')

In [ ]:
model.fit(x=train_x,y=train_y,batch_size=32,epochs=30,shuffle=True,validation_data=(train_dev_x,train_dev_y),callbacks = [TrainValTensorBoard(write_graph=False)])

Train on 53999 samples, validate on 6000 samples
Epoch 1/30
53999/53999 [==============================] - 13s 244us/step - loss: 0.4607 - acc: 0.8964 - val_loss: 0.3072 - val_acc: 0.9383
Epoch 2/30
53999/53999 [==============================] - 9s 170us/step - loss: 0.2783 - acc: 0.9421 - val_loss: 0.2291 - val_acc: 0.9558
Epoch 3/30
53999/53999 [==============================] - 9s 172us/step - loss: 0.2361 - acc: 0.9516 - val_loss: 0.2150 - val_acc: 0.9568
Epoch 4/30
53999/53999 [==============================] - 9s 169us/step - loss: 0.2093 - acc: 0.9574 - val_loss: 0.2080 - val_acc: 0.9582
Epoch 5/30
53999/53999 [==============================] - 9s 174us/step - loss: 0.1928 - acc: 0.9596 - val_loss: 0.1809 - val_acc: 0.9625
Epoch 6/30
53999/53999 [==============================] - 9s 169us/step - loss: 0.1802 - acc: 0.9631 - val_loss: 0.1762 - val_acc: 0.9648
Epoch 7/30
53999/53999 [==============================] - 9s 169us/step - loss: 0.1728 - acc: 0.9636 - val_loss: 0.1711 - 

In [ ]:
print("Dev Data-->",model.evaluate(x=dev_x, y=dev_y, batch_size=32)[1])

In [ ]:
print("Test Data -->",model.evaluate(x=test_x, y=test_y, batch_size=32)[1])

In [ ]:
print("Train Data -->",model.evaluate(x=train_x, y=train_y, batch_size=32)[1])

In [20]:
class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir='/tmp/log/', **kwargs):
        self.val_log_dir = os.path.join(log_dir, 'validation')
        training_log_dir = os.path.join(log_dir, 'training')
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

    def set_model(self, model):
        if context.executing_eagerly():
            self.val_writer = tensorflow.contrib.summary.create_file_writer(self.val_log_dir)
        else:
            self.val_writer = tensorflow.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def _write_custom_summaries(self, step, logs=None):
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if 'val_' in k}
        if context.executing_eagerly():
            with self.val_writer.as_default(), tensorflow.contrib.summary.always_record_summaries():
                for name, value in val_logs.items():
                    tensorflow.contrib.summary.scalar(name, value.item(), step=step)
        else:
            for name, value in val_logs.items():
                summary = tensorflow.Summary()
                summary_value = summary.value.add()
                summary_value.simple_value = value.item()
                summary_value.tag = name
                self.val_writer.add_summary(summary, step)
        self.val_writer.flush()

        logs = {k: v for k, v in logs.items() if not 'val_' in k}
        super(TrainValTensorBoard, self)._write_custom_summaries(step, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()